#Prediksi Harga Rumah Jaksel

In [50]:
from google.colab import files
filehome = files.upload()

Saving archive.zip to archive (1).zip


In [51]:
!unzip 'archive.zip'

Archive:  archive.zip
replace DATA RUMAH.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DATA RUMAH.xlsx         
replace HARGA RUMAH JAKSEL.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: HARGA RUMAH JAKSEL.xlsx  


In [52]:
import pandas as pd
data = pd.read_excel('HARGA RUMAH JAKSEL.xlsx', sheet_name = 'Sheet1', skiprows=1)

In [53]:
data.head()

,HARGA,LT,LB,JKT,JKM,GRS,KOTA
0,28000000000,1100,700,5,6,ADA,JAKSEL
1,19000000000,824,800,4,4,ADA,JAKSEL
2,4700000000,500,400,4,3,ADA,JAKSEL
3,4900000000,251,300,5,4,ADA,JAKSEL
4,28000000000,1340,575,4,5,ADA,JAKSEL


##Exploratory Data Analysis

In [54]:
pd.DataFrame(data['HARGA'].describe())

,HARGA
count,1.001000e+03
mean,1.747472e+10
std,2.079548e+10
min,4.300000e+08
25%,6.750000e+09
50%,1.350000e+10
75%,2.000000e+10
max,2.500000e+11


### Analisis Bivariat

In [55]:
import plotly.express as px

#### harga dan luas tanah

In [56]:
px.histogram(
    data_frame = data, x = 'LT', y = 'HARGA',
    nbins = 5, histfunc = 'avg', color_discrete_sequence=['orange'],
    title = 'Perbandingan Harga dengan Luas Tanah'
    )

#### Harga dengan Luas Bangunan

In [57]:
px.histogram (
    data_frame = data, x = 'LB', y = 'HARGA',
    nbins = 5, histfunc = 'avg', color_discrete_sequence = ['green'],
    title = 'Perbandingan Harga dengan Luas Bangunan'
)

### Analisis Unvariat

In [58]:
pd.DataFrame(data['GRS'].value_counts().reset_index())

,index,GRS
0,ADA,779
1,TIDAK ADA,222


In [59]:
px.pie(
    data_frame = data['GRS'].value_counts().reset_index(), values = 'GRS',
    names ='index', title = 'Jumlah Rumah Jaksel yang punya Garasi'
    )

In [60]:
pd.DataFrame(data['JKT'].describe())

,JKT
count,1001.000000
mean,4.457542
std,2.004606
min,1.000000
25%,4.000000
50%,4.000000
75%,5.000000
max,27.000000


In [63]:
dataKamarTidur = pd.DataFrame(
    data['JKT'].value_counts().reset_index().rename(columns = {'index':'JKT','JKT':'Banyak Rumah'})
    )
px.bar(
    data_frame = dataKamarTidur, x = 'JKT', y = 'Banyak Rumah',
    title = 'Jumlah Kamar Tidur Rumah Jaksel'
    )

In [64]:
data['JKM'].describe()

count    1001.000000
mean        3.940060
std         1.903261
min         1.000000
25%         3.000000
50%         4.000000
75%         4.000000
max        27.000000
Name: JKM, dtype: float64

In [65]:
dataKamarMandi = pd.DataFrame(
    data['JKM'].value_counts().reset_index().rename(columns={'index':'JKM','JKM':'Banyak Rumah'})
    )
px.bar(data_frame = dataKamarMandi, x = 'JKM', y = 'Banyak Rumah', title='Jumlah Kamar Mandi')

In [66]:
data['KOTA'].value_counts()

JAKSEL    1001
Name: KOTA, dtype: int64

## Preprosessing

In [67]:
data.drop(columns='KOTA',inplace=True,axis=1)

In [69]:
data.GRS = data.GRS.map({'ADA':1,'TIDAK ADA':0})

In [70]:
data.head()

,HARGA,LT,LB,JKT,JKM,GRS
0,28000000000,1100,700,5,6,1
1,19000000000,824,800,4,4,1
2,4700000000,500,400,4,3,1
3,4900000000,251,300,5,4,1
4,28000000000,1340,575,4,5,1


In [71]:
data.shape

(1001, 6)

In [72]:
data = data[(data.LT>data.LT.quantile(0.01))&(data.LT<data.LT.quantile(0.99))]
data = data[(data.LB>data.LB.quantile(0.01))&(data.LB<data.LB.quantile(0.99))]
data = data[(data.JKT>data.JKT.quantile(0.01))&(data.JKT<data.JKT.quantile(0.99))]
data = data[(data.JKM>data.JKM.quantile(0.01))&(data.JKM<data.JKM.quantile(0.99))]
data = data[(data.HARGA>data.HARGA.quantile(0.01))&(data.HARGA<data.HARGA.quantile(0.99))]

In [73]:
data.shape

(823, 6)

##Prediksi

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [75]:
X = data.iloc[:,1:]
X

,LT,LB,JKT,JKM,GRS
0,1100,700,5,6,1
1,824,800,4,4,1
2,500,400,4,3,1
3,251,300,5,4,1
4,1340,575,4,5,1
...,...,...,...,...,...
993,169,215,4,4,1
996,488,550,6,5,1
997,209,270,4,4,1
998,692,400,4,3,0


In [76]:
y = data.iloc[:,0]
y

0      28000000000
1      19000000000
2       4700000000
3       4900000000
4      28000000000
          ...     
993     3500000000
996    16000000000
997     4500000000
998    29000000000
999     1700000000
Name: HARGA, Length: 823, dtype: int64

In [77]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [78]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
hasil = model.predict(X_test)

In [79]:
hasil

array([1.77120000e+10, 3.16140000e+09, 6.52450000e+09, 5.47700000e+09,
       1.52310000e+10, 2.25170000e+10, 2.15200000e+09, 1.21417000e+10,
       4.41300000e+09, 2.84600000e+10, 2.27389000e+10, 1.63005250e+10,
       2.81920000e+10, 1.41950000e+10, 1.55797500e+10, 1.93900000e+10,
       1.71213429e+10, 1.45660000e+10, 7.11700000e+09, 3.96200000e+10,
       3.48720000e+09, 1.30010000e+10, 9.68000000e+09, 2.97895000e+10,
       1.01560000e+10, 2.45319000e+10, 2.19580000e+10, 1.28825000e+10,
       5.13130000e+09, 1.11055167e+10, 2.32367500e+10, 2.66050000e+10,
       2.37110000e+10, 2.37020000e+10, 1.95638333e+10, 2.31809000e+10,
       1.07580000e+10, 5.34752381e+10, 4.25080000e+09, 1.30340000e+10,
       6.78246667e+09, 1.15890000e+10, 2.33985000e+10, 2.01625000e+10,
       9.11050000e+09, 3.96200000e+10, 2.39390000e+10, 5.81800000e+09,
       1.24530000e+10, 2.47780000e+09, 4.56240000e+10, 2.50760000e+10,
       6.49210000e+09, 1.69740000e+10, 1.61852333e+10, 1.65285000e+10,
      

In [80]:
def prediksi(LT,LB,JKT,JKM,GRS):
    predict = pd.DataFrame()
    predict['LT'] = [LT]
    predict['LB'] = [LB]
    predict['JKT'] = [JKT]
    predict['JKM'] = [JKM]
    predict['GRS'] = [GRS]
    print(model.predict(predict))

In [81]:
prediksi(500,600,2,2,1)

[1.69714167e+10]


##Deployment

In [82]:
import  pickle
with open('prediksi_harga_rumah.pkl','wb') as file:
    pickle.dump(model,file)

In [83]:
%%writefile app.py
import pickle
import pandas as pd
import streamlit as st

with open ("prediksi_harga_rumah.pkl", "rb") as f:
    model = pickle.load(f)

def prediksi(LT,LB,JKT,JKM,GRS):
    predict = pd.DataFrame()
    predict['Luas Tanah'] = [LT]
    predict['Luas Bangunan'] = [LB]
    predict['Jumlah Kamar Tidur'] = [JKT]
    predict['Jumlah Kamar Mandi'] = [JKM]
    predict['Garasi'] = [GRS]
    return(model.predict(predict)[0])   

lt = st.number_input("Luas Tanah")
lb = st.number_input("Luas Bangunan")
jkt = st.number_input("Jumlah Kamar Tidur")
jkm = st.number_input("Jumlah Kamar Mandi")
opt = st.selectbox(
    'Garasi',
    ('Ada', 'Tidak ada')
)

grs = 0
if(opt == 'Ada') :
    grs = 1
elif (opt == 'Tidak ada'):
    grs = 0
    
    
if(st.button('Predict')) :
    st.write("Harga rumah impianmu adalah Rp{:,}".format(prediksi(lt,lb,jkt,jkm,grs)))

Writing app.py


In [84]:
%%writefile requirements.txt
scikit-learn

Writing requirements.txt
